In [23]:
class Person:
    pass

In [26]:
steve = Person()

In [28]:
type(steve)

__main__.Person

In [29]:
type([])

list

In [30]:
type(id(steve))

int

In [31]:
id(steve)

4515742640

In [32]:
type(Person)

type

In [33]:
isinstance(Person, type)

True

In [34]:
isinstance(Person(), type)

False

In [36]:
isinstance(steve, Person), isinstance(steve, object)  # Person(object)

(True, True)

In [37]:
issubclass(Person, object), issubclass(Person, dict)

(True, False)

In [38]:
isinstance(steve, (dict, list, Person)), isinstance(steve, (str, int, bool))

(True, False)

In [39]:
isinstance(object, type)

True

In [40]:
isinstance(type, object)

True

In [41]:
issubclass(Person, type), issubclass(object, type)

(False, False)

In [42]:
isinstance(Person, type), isinstance(object, type)

(True, True)

In [43]:
isinstance(Person, object), isinstance(type, object)

(True, True)

In [45]:
isinstance(Person, type), issubclass(type, object)

(True, True)

In [46]:
isinstance(Person, object), issubclass(Person, object)

(True, True)

In [50]:
isinstance(type, type), isinstance(object, object)

(True, True)

In [51]:
class Foo:
    name = "foo_name"

    def __init__(self, val):
        self.val = val

In [53]:
foo_inst = Foo(33)

In [54]:
foo_inst.__dict__

{'val': 33}

In [55]:
Foo.__dict__

mappingproxy({'__module__': '__main__',
              'name': 'foo_name',
              '__init__': <function __main__.Foo.__init__(self, val)>,
              '__dict__': <attribute '__dict__' of 'Foo' objects>,
              '__weakref__': <attribute '__weakref__' of 'Foo' objects>,
              '__doc__': None})

In [56]:
Bar = type("Bar", (Foo,), dict(attr=100))

In [57]:
Bar.__dict__

mappingproxy({'attr': 100, '__module__': '__main__', '__doc__': None})

In [58]:
bar_inst = Bar(99)

In [59]:
bar_inst.__dict__

{'val': 99}

In [65]:
Baz = type("Baz", (), dict(__init__=lambda self, val: setattr(self, "val", val)))

baz_inst = Baz(77)

In [66]:
baz_inst.val

77

In [67]:
Baz.mro()

[__main__.Baz, object]

In [68]:
baz_inst.__dict__

{'val': 77}

In [69]:
class AMeta(type):
    def __new__(mcs, name, bases, classdict, **kwargs):
        cls = super().__new__(mcs, name, bases, classdict)
        print('Meta __new__', cls)

        return cls

    def __init__(cls, name, bases, classdict, **kwargs):
        print('Meta __init__', cls, name, bases, classdict)
        super().__init__(name, bases, classdict, **kwargs)

    def __call__(cls, *args, **kwargs):
        print('Meta __call__', cls, args, kwargs)
        return super().__call__(*args, **kwargs)

    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print('Meta __prepare__', mcs, name, bases, kwargs)
        
        return {'added_attr_a': 2, 'added_attr_b': 99}

In [80]:
class Person(metaclass=AMeta):
    age = 99

    def __new__(cls, *args, **kwargs):
        inst = super().__new__(cls)
        print('Person __new__', inst)

        return inst

    def __init__(self, name):
        print("Person.__init__", name)
        self.name = name

Meta __prepare__ <class '__main__.AMeta'> Person () {}
Meta __new__ <class '__main__.Person'>
Meta __init__ <class '__main__.Person'> Person () {'added_attr_a': 2, 'added_attr_b': 99, '__module__': '__main__', '__qualname__': 'Person', 'age': 99, '__new__': <function Person.__new__ at 0x10cd8aca0>, '__init__': <function Person.__init__ at 0x10cd88040>, '__classcell__': <cell at 0x10dd8cbe0: AMeta object at 0x7f899aca1b10>}


In [74]:
Person

__main__.Person

In [81]:
jd = Person("JD")

Meta __call__ <class '__main__.Person'> ('JD',) {}
Person __new__ <__main__.Person object at 0x10dd8e630>
Person.__init__ JD


In [82]:
jd.__dict__

{'name': 'JD'}

In [83]:
Student = AMeta("Student", (Person,), {"school": None})

Meta __new__ <class '__main__.Student'>
Meta __init__ <class '__main__.Student'> Student (<class '__main__.Person'>,) {'school': None}


In [85]:
Student, Student.__dict__

(__main__.Student,
 mappingproxy({'school': None, '__module__': '__main__', '__doc__': None}))

In [86]:
st = Student("Lorem")

Meta __call__ <class '__main__.Student'> ('Lorem',) {}
Person __new__ <__main__.Student object at 0x10d26d4f0>
Person.__init__ Lorem


In [88]:
AMeta([1, 2])

TypeError: AMeta.__new__() missing 2 required positional arguments: 'bases' and 'classdict'

In [92]:
class SingleMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        print("SingleMeta.__call__", cls)

        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)

        return cls._instances[cls]
            

class Animal(metaclass=SingleMeta):
    def __init__(self, name):
        print("Animal.__init__", name)
        self.name = name


tiger = Animal("tiger")
print(f"{id(tiger)=}")

dragon = Animal("dragon")

print(f"{id(tiger)=}")
print(tiger is dragon, tiger.name, tiger.__dict__)

SingleMeta.__call__ <class '__main__.Animal'>
Animal.__init__ tiger
id(tiger)=4527191680
SingleMeta.__call__ <class '__main__.Animal'>
id(tiger)=4527191680
True tiger {'name': 'tiger'}


In [93]:
class Divan(metaclass=SingleMeta):
    def __init__(self, name):
        print("Divan.__init__", name)
        self.name = name


In [95]:
d1 = Divan("divan")
d2 = Divan("sofa")

d1 is d2

SingleMeta.__call__ <class '__main__.Divan'>
Divan.__init__ divan
SingleMeta.__call__ <class '__main__.Divan'>


True

In [96]:
Animal("tiger")

SingleMeta.__call__ <class '__main__.Animal'>


In [97]:
Animal.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Animal.__init__(self, name)>,
              '__dict__': <attribute '__dict__' of 'Animal' objects>,
              '__weakref__': <attribute '__weakref__' of 'Animal' objects>,
              '__doc__': None})

In [99]:
Animal._instances, Animal._instances is Divan._instances

({__main__.Animal: <__main__.Animal at 0x10dd77680>,
  __main__.Divan: <__main__.Divan at 0x10dd8e450>},
 True)

In [101]:
import abc

In [106]:
class Person(metaclass=abc.ABCMeta):
    
    @abc.abstractmethod
    def eat(self, food):
        print("Person.eat", food)
        return 99

    @abc.abstractmethod
    def walk(self):
        pass

    def study(self):
        print("Person.study")
        self.eat("apple")
        self.walk()

In [107]:
steve = Person()

TypeError: Can't instantiate abstract class Person without an implementation for abstract methods 'eat', 'walk'

In [108]:
class Student(Person):
    def walk(self):
        print("Student.walk")

In [109]:
stue = Student()

TypeError: Can't instantiate abstract class Student without an implementation for abstract method 'eat'

In [126]:
class HSEStudent(Student):
    def eat(self, food):
        print("HSEStudent.eat", food)

        return super().eat(food)

In [127]:
hse = HSEStudent()

In [122]:
hse.__dict__

{}

In [128]:
print(hse.eat("apple"))

HSEStudent.eat apple
Person.eat apple
99


In [115]:
hse.walk()

Student.walk


In [119]:
hse.study()

Person.study
HSEStudent.eat apple
Student.walk


In [129]:
Person.register?

Signature: Person.register(subclass)
Docstring:
Register a virtual subclass of an ABC.

Returns the subclass, to allow usage as a class decorator.
File:      ~/projects/hse_deep_python_autumn_2024/lesson-05/<frozen abc>
Type:      method

In [ ]:
Person.register

In [130]:
isinstance("not_person", Person), issubclass(str, Person)

(False, False)

In [131]:
Person.register(str)

str

In [132]:
isinstance("not_person", Person), issubclass(str, Person)

(True, True)

In [133]:
from collections import abc as bc

In [134]:
bc.Hashable

collections.abc.Hashable

In [135]:
isinstance("qwerty", bc.Hashable)

True

In [136]:
isinstance([1, 2, 3], bc.Hashable)

False

In [137]:
bc.Iterable, bc.Iterator

(collections.abc.Iterable, collections.abc.Iterator)

In [138]:
class SomeIterable:
    def __iter__(self):
        pass


class SomeIterator:
    def __next__(self):
        pass


class Some(SomeIterable, SomeIterator):
    pass

In [141]:
for cls in (SomeIterable, SomeIterator, Some):
    print(f"{cls=}, {issubclass(cls, bc.Iterable)=}, {issubclass(cls, bc.Iterator)=}")

cls=<class '__main__.SomeIterable'>, issubclass(cls, bc.Iterable)=True, issubclass(cls, bc.Iterator)=False
cls=<class '__main__.SomeIterator'>, issubclass(cls, bc.Iterable)=False, issubclass(cls, bc.Iterator)=False
cls=<class '__main__.Some'>, issubclass(cls, bc.Iterable)=True, issubclass(cls, bc.Iterator)=True


In [194]:
class Predictable(abc.ABC):
    @abc.abstractmethod
    def predict(self):
        print("Predictable.predict")

    @classmethod
    def __subclasshook__(cls, C):
        print(f"Predictable.__subclasshook__, {cls=}, {C=}")

        if cls is not Predictable:
            return NotImplemented

        name = "predict"

        for class_ in C.__mro__:
            if name in class_.__dict__ and callable(class_.__dict__[name]):
                return True

        return NotImplemented

In [182]:
class LLama:
    def predict(self, data):
        return 42


class LightTraffic:
    predict = "green"


llm = LLama()
light = LightTraffic()

In [183]:
isinstance(llm, Predictable), issubclass(LLama, Predictable)

Predictable.__subclasshook__, cls=<class '__main__.Predictable'>, C=<class '__main__.LLama'>


(True, True)

In [184]:
issubclass(LLama, Predictable)

True

In [185]:
isinstance(light, Predictable), issubclass(LightTraffic, Predictable)

Predictable.__subclasshook__, cls=<class '__main__.Predictable'>, C=<class '__main__.LightTraffic'>


(False, False)

In [186]:
type(steve)

__main__.Person

In [187]:
isinstance(steve, Predictable), issubclass(Person, Predictable)

Predictable.__subclasshook__, cls=<class '__main__.Predictable'>, C=<class '__main__.Person'>
Predictable.__subclasshook__, cls=<class '__main__.Predictable'>, C=<class '__main__.Person'>


(False, False)

In [192]:
class TrainPredictable(Predictable):
    @abc.abstractmethod
    def train(self):
        pass

    # @classmethod
    # def __subclasshook__(cls, C):
    #     print(f"TrainPredictable.__subclasshook__, {cls=}, {C=}")

    #     if cls is not TrainPredictable:
    #         return NotImplemented

    #     name = "predict"

    #     for class_ in C.__mro__:
    #         if name in class_.__dict__ and callable(class_.__dict__[name]):
    #             return True

    #     return NotImplemented

class LLamaTrain:
    def predict(self, data):
        return 42

    def train(self):
        pass


model = LLamaTrain()

In [193]:
isinstance(model, TrainPredictable)

Predictable.__subclasshook__, cls=<class '__main__.TrainPredictable'>, C=<class '__main__.LLamaTrain'>


True

In [195]:
0.1, 0.2

(0.1, 0.2)

In [196]:
0.1 + 0.2

0.30000000000000004

In [197]:
0.1 + 0.2 == 0.3

False

In [201]:
a = 0.1 + 0.2
b = 0.3

print(a == b)

print(abs(a - b) < 10e-14)

False
True


In [202]:
import math

In [203]:
math.isclose(a, b)

True

In [212]:
math.isclose(a, b, rel_tol=10e-20)

False

In [215]:
math.isclose(a, b, abs_tol=10e-100)

True

In [217]:
a.as_integer_ratio()

(1351079888211149, 4503599627370496)

In [218]:
from decimal import Decimal
from fractions import Fraction

In [219]:
d1 = Decimal("0.1")
d2 = Decimal("0.2")

d1 + d2

Decimal('0.3')

In [220]:
d1 = Decimal(0.1)
d2 = Decimal("0.2")

d1 + d2

Decimal('0.3000000000000000055511151231')

In [221]:
d1

Decimal('0.1000000000000000055511151231257827021181583404541015625')

In [222]:
d1 = Decimal("0.1")
d2 = Decimal("0.2")

d1 + d2

Decimal('0.3')

In [223]:
d1 / d2

Decimal('0.5')

In [224]:
Decimal(1) / Decimal(7)

Decimal('0.1428571428571428571428571429')

In [229]:
f1 = Fraction(1, 7)
f2 = Fraction(3, 14)

In [228]:
f1.numerator, f1.denominator, f1

(1, 7, Fraction(1, 7))

In [230]:
f1 + f2

Fraction(5, 14)

In [231]:
5 / 14

0.35714285714285715

In [232]:
float(f1 + f2)

0.35714285714285715